# 📈 Time Series Analysis: NYC School Performance Trends

**Objective:** Analyze how school performance has changed over time

**Created:** October 6, 2025

## What We'll Do:
1. Fetch historical SAT score data from NYC Open Data
2. Analyze trends over multiple years
3. Identify schools with improving/declining performance
4. Examine borough-level trends
5. Forecast future performance

In [ ]:
# Imports
import sys
from pathlib import Path
from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Add project root to path
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

from scripts.utils import load_csv, save_csv

# Configuration
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
sns.set_palette('husl')

print("✅ Imports successful!")
print(f"📁 Project root: {PROJECT_ROOT}")

## 1. Load Current Data

First, let's load the most recent SAT data we have.

In [ ]:
# Load current year data
try:
    current_data = load_csv("zt9s-n5aj_20251006_114228.csv")
    print(f"✅ Loaded current data: {len(current_data)} schools")
except FileNotFoundError:
    print("⚠️ Current data file not found. Using processed data.")
    current_data = load_csv("nyc_education_analyzed.csv", subfolder="processed")

# Convert SAT scores to numeric
for col in ['mathematics_mean', 'critical_reading_mean', 'writing_mean', 'number_of_test_takers']:
    if col in current_data.columns:
        current_data[col] = pd.to_numeric(current_data[col], errors='coerce')

# Calculate total score
current_data['total_score'] = (
    current_data['mathematics_mean'] + 
    current_data['critical_reading_mean'] + 
    current_data['writing_mean']
)

# Extract or add year
current_data['year'] = 2024  # Most recent year in dataset

print(f"\nData shape: {current_data.shape}")
print(f"\nSample data:")
current_data[['dbn', 'school_name', 'total_score', 'year']].head()

## 2. Fetch Historical Data (Optional)

To do true time series analysis, we would fetch historical data from NYC Open Data.

**Note:** For this example, we'll simulate historical data. In production, you would:
```python
# Fetch data for multiple years
!python ../scripts/fetch_data_gov.py zt9s-n5aj --domain data.cityofnewyork.us --limit 5000
```

In [ ]:
# Check for other SAT data files in raw directory
data_dir = PROJECT_ROOT / 'data' / 'raw'
sat_files = list(data_dir.glob('zt9s-n5aj*.csv'))

print(f"Found {len(sat_files)} SAT data file(s):")
for f in sat_files:
    print(f"  • {f.name}")

# If we only have one file, simulate historical data for demonstration
if len(sat_files) <= 1:
    print("\n📝 Simulating historical data for demonstration...")
    
    # Create simulated data for 5 previous years
    years = [2019, 2020, 2021, 2022, 2023, 2024]
    historical_dfs = []
    
    for year in years:
        df_year = current_data.copy()
        df_year['year'] = year
        
        # Simulate score changes over time (random walk with slight upward trend)
        if year < 2024:
            years_diff = 2024 - year
            # Add random variation and slight trend
            trend = np.random.normal(0, 10, len(df_year))
            year_effect = np.random.normal(-years_diff * 2, 15, len(df_year))  # Slight downward trend in past
            
            df_year['mathematics_mean'] = np.clip(df_year['mathematics_mean'] + year_effect + trend, 200, 800)
            df_year['critical_reading_mean'] = np.clip(df_year['critical_reading_mean'] + year_effect + trend, 200, 800)
            df_year['writing_mean'] = np.clip(df_year['writing_mean'] + year_effect + trend, 200, 800)
            df_year['total_score'] = df_year['mathematics_mean'] + df_year['critical_reading_mean'] + df_year['writing_mean']
        
        historical_dfs.append(df_year)
    
    # Combine all years
    time_series_df = pd.concat(historical_dfs, ignore_index=True)
    
    print(f"✅ Created simulated time series data: {len(time_series_df)} records across {len(years)} years")
else:
    # Load multiple files if available
    print("\n📊 Loading multiple historical files...")
    historical_dfs = []
    for f in sat_files:
        df = pd.read_csv(f)
        # Try to extract year from filename or data
        historical_dfs.append(df)
    time_series_df = pd.concat(historical_dfs, ignore_index=True)

print(f"\nTime series shape: {time_series_df.shape}")
print(f"Years in dataset: {sorted(time_series_df['year'].unique())}")

## 3. Overall Performance Trends

In [ ]:
# Calculate yearly averages
yearly_avg = time_series_df.groupby('year').agg({
    'total_score': ['mean', 'median', 'std'],
    'mathematics_mean': 'mean',
    'critical_reading_mean': 'mean',
    'writing_mean': 'mean',
    'number_of_test_takers': 'sum'
}).round(1)

print("📊 NYC Average SAT Scores Over Time\n" + "="*70)
print(yearly_avg)

# Create interactive plot
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Average Total SAT Score', 'SAT Sections Over Time', 
                    'Total Test Takers', 'Score Variability (Std Dev)'),
    specs=[[{'secondary_y': False}, {'secondary_y': False}],
           [{'secondary_y': False}, {'secondary_y': False}]]
)

years = sorted(time_series_df['year'].unique())
yearly_avg_flat = time_series_df.groupby('year')['total_score'].mean()

# Plot 1: Total score trend
fig.add_trace(
    go.Scatter(x=years, y=yearly_avg_flat.values, mode='lines+markers', 
               name='Avg Total SAT', line=dict(color='steelblue', width=3)),
    row=1, col=1
)

# Plot 2: Individual sections
sections = {
    'Math': time_series_df.groupby('year')['mathematics_mean'].mean(),
    'Reading': time_series_df.groupby('year')['critical_reading_mean'].mean(),
    'Writing': time_series_df.groupby('year')['writing_mean'].mean()
}
colors = {'Math': 'green', 'Reading': 'purple', 'Writing': 'orange'}

for section, data in sections.items():
    fig.add_trace(
        go.Scatter(x=years, y=data.values, mode='lines+markers', 
                   name=section, line=dict(color=colors[section])),
        row=1, col=2
    )

# Plot 3: Test takers
total_takers = time_series_df.groupby('year')['number_of_test_takers'].sum()
fig.add_trace(
    go.Bar(x=years, y=total_takers.values, name='Total Test Takers', marker_color='coral'),
    row=2, col=1
)

# Plot 4: Variability
std_dev = time_series_df.groupby('year')['total_score'].std()
fig.add_trace(
    go.Scatter(x=years, y=std_dev.values, mode='lines+markers', 
               name='Std Dev', line=dict(color='red', width=2)),
    row=2, col=2
)

fig.update_layout(height=800, showlegend=True, title_text="NYC SAT Performance Trends Over Time")
fig.update_xaxes(title_text="Year")
fig.show()

# Calculate change
if len(years) > 1:
    first_year_avg = yearly_avg_flat.iloc[0]
    last_year_avg = yearly_avg_flat.iloc[-1]
    total_change = last_year_avg - first_year_avg
    pct_change = (total_change / first_year_avg) * 100
    
    print(f"\n📈 Overall Trend ({years[0]} to {years[-1]}):")
    print(f"  • Starting average: {first_year_avg:.1f}")
    print(f"  • Ending average: {last_year_avg:.1f}")
    print(f"  • Total change: {total_change:+.1f} points ({pct_change:+.1f}%)")

## 4. Borough-Level Trends

In [ ]:
# Add borough mapping if not present
borough_names = {'M': 'Manhattan', 'X': 'Bronx', 'K': 'Brooklyn', 'Q': 'Queens', 'R': 'Staten Island'}

if 'borough_name' not in time_series_df.columns and 'boro' in time_series_df.columns:
    time_series_df['borough_name'] = time_series_df['boro'].map(borough_names)
elif 'borough_name' not in time_series_df.columns:
    time_series_df['borough_name'] = time_series_df['dbn'].str[2].map(borough_names)

# Calculate borough trends
borough_trends = time_series_df.groupby(['year', 'borough_name'])['total_score'].mean().reset_index()

# Plot borough trends
fig = px.line(
    borough_trends,
    x='year',
    y='total_score',
    color='borough_name',
    markers=True,
    title='Average SAT Scores by Borough Over Time',
    labels={'total_score': 'Average SAT Score', 'year': 'Year', 'borough_name': 'Borough'},
    line_shape='spline'
)

fig.update_layout(height=600, hovermode='x unified')
fig.show()

# Calculate borough growth rates
print("\n🗽 Borough Performance Changes\n" + "="*70)

for borough in borough_trends['borough_name'].unique():
    if pd.notna(borough):
        borough_data = borough_trends[borough_trends['borough_name'] == borough].sort_values('year')
        if len(borough_data) > 1:
            first_score = borough_data.iloc[0]['total_score']
            last_score = borough_data.iloc[-1]['total_score']
            change = last_score - first_score
            pct_change = (change / first_score) * 100
            
            emoji = "📈" if change > 0 else "📉" if change < 0 else "➡️"
            print(f"{emoji} {borough:15} {first_score:6.1f} → {last_score:6.1f} ({change:+6.1f}, {pct_change:+5.1f}%)")

## 5. Individual School Trends

Identify schools with the most improvement and decline.

In [ ]:
# Calculate change for each school
school_first_last = time_series_df.groupby('dbn').agg({
    'school_name': 'first',
    'total_score': ['first', 'last'],
    'year': ['min', 'max']
}).reset_index()

school_first_last.columns = ['dbn', 'school_name', 'first_score', 'last_score', 'first_year', 'last_year']
school_first_last['change'] = school_first_last['last_score'] - school_first_last['first_score']
school_first_last['pct_change'] = (school_first_last['change'] / school_first_last['first_score']) * 100

# Remove schools with no change (only one year of data)
school_changes = school_first_last[school_first_last['first_year'] != school_first_last['last_year']].copy()

print("🏆 Top 10 Most Improved Schools\n" + "="*70)
top_improvers = school_changes.nlargest(10, 'change')[['school_name', 'first_score', 'last_score', 'change', 'pct_change']]
print(top_improvers.to_string(index=False))

print("\n📉 Top 10 Declining Schools\n" + "="*70)
top_decliners = school_changes.nsmallest(10, 'change')[['school_name', 'first_score', 'last_score', 'change', 'pct_change']]
print(top_decliners.to_string(index=False))

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Distribution of changes
axes[0].hist(school_changes['change'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(0, color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel('Change in SAT Score')
axes[0].set_ylabel('Number of Schools')
axes[0].set_title('Distribution of Score Changes')
axes[0].grid(alpha=0.3)

# Scatter: first vs last scores
axes[1].scatter(school_changes['first_score'], school_changes['last_score'], alpha=0.5)
axes[1].plot([800, 2400], [800, 2400], 'r--', label='No Change')
axes[1].set_xlabel('First Year Score')
axes[1].set_ylabel('Last Year Score')
axes[1].set_title('School Performance: First vs Last Year')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'data' / 'processed' / 'school_trends_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# Summary statistics
improving = (school_changes['change'] > 0).sum()
declining = (school_changes['change'] < 0).sum()
stable = (school_changes['change'] == 0).sum()

print(f"\n📊 Summary:")
print(f"  • Improving schools: {improving} ({improving/len(school_changes)*100:.1f}%)")
print(f"  • Declining schools: {declining} ({declining/len(school_changes)*100:.1f}%)")
print(f"  • Stable schools: {stable} ({stable/len(school_changes)*100:.1f}%)")
print(f"  • Average change: {school_changes['change'].mean():.1f} points")
print(f"  • Median change: {school_changes['change'].median():.1f} points")

## 6. Year-over-Year Growth Analysis

In [ ]:
# Calculate year-over-year changes
time_series_sorted = time_series_df.sort_values(['dbn', 'year'])
time_series_sorted['prev_year_score'] = time_series_sorted.groupby('dbn')['total_score'].shift(1)
time_series_sorted['yoy_change'] = time_series_sorted['total_score'] - time_series_sorted['prev_year_score']
time_series_sorted['yoy_pct'] = (time_series_sorted['yoy_change'] / time_series_sorted['prev_year_score']) * 100

# Aggregate by year
yoy_summary = time_series_sorted.groupby('year').agg({
    'yoy_change': ['mean', 'median', 'std'],
    'yoy_pct': ['mean', 'median']
}).round(2)

print("📅 Year-over-Year Changes\n" + "="*70)
print(yoy_summary)

# Plot YoY changes
yoy_by_year = time_series_sorted.groupby('year')['yoy_change'].mean().dropna()

fig = go.Figure()
fig.add_trace(go.Bar(
    x=yoy_by_year.index,
    y=yoy_by_year.values,
    marker_color=['green' if x > 0 else 'red' for x in yoy_by_year.values],
    text=[f"{x:+.1f}" for x in yoy_by_year.values],
    textposition='outside'
))

fig.update_layout(
    title='Average Year-over-Year Change in SAT Scores',
    xaxis_title='Year',
    yaxis_title='Average Change (points)',
    height=500
)
fig.add_hline(y=0, line_dash='dash', line_color='black')
fig.show()

## 7. Volatility Analysis

Which schools have the most consistent vs volatile performance?

In [ ]:
# Calculate standard deviation for each school across years
school_volatility = time_series_df.groupby('dbn').agg({
    'school_name': 'first',
    'total_score': ['mean', 'std', 'count'],
    'borough_name': 'first'
}).reset_index()

school_volatility.columns = ['dbn', 'school_name', 'avg_score', 'std_dev', 'years_count', 'borough']

# Only include schools with data for multiple years
school_volatility = school_volatility[school_volatility['years_count'] > 1]

# Calculate coefficient of variation (CV)
school_volatility['cv'] = (school_volatility['std_dev'] / school_volatility['avg_score']) * 100

print("🎯 Most Consistent Schools (Low Volatility)\n" + "="*70)
consistent = school_volatility.nsmallest(10, 'std_dev')[['school_name', 'avg_score', 'std_dev', 'cv']]
print(consistent.to_string(index=False))

print("\n🎢 Most Volatile Schools\n" + "="*70)
volatile = school_volatility.nlargest(10, 'std_dev')[['school_name', 'avg_score', 'std_dev', 'cv']]
print(volatile.to_string(index=False))

# Visualize
fig = px.scatter(
    school_volatility,
    x='avg_score',
    y='std_dev',
    color='borough',
    hover_data=['school_name'],
    title='School Performance: Average vs Volatility',
    labels={'avg_score': 'Average SAT Score', 'std_dev': 'Standard Deviation (Volatility)', 'borough': 'Borough'},
    size='std_dev',
    size_max=15
)
fig.update_layout(height=600)
fig.show()

# Borough volatility comparison
if 'borough' in school_volatility.columns:
    borough_vol = school_volatility.groupby('borough')['std_dev'].mean().sort_values(ascending=False)
    print("\n📊 Average Volatility by Borough:")
    for borough, vol in borough_vol.items():
        if pd.notna(borough):
            print(f"  • {borough:15} {vol:.1f} points")

## 8. Save Results

In [ ]:
# Save trend analysis results
if len(school_changes) > 0:
    save_csv(school_changes, 'school_trends_summary.csv', subfolder='processed', index=False)
    print("✅ Saved: school_trends_summary.csv")

if len(school_volatility) > 0:
    save_csv(school_volatility, 'school_volatility_analysis.csv', subfolder='processed', index=False)
    print("✅ Saved: school_volatility_analysis.csv")

# Save borough trends
save_csv(borough_trends, 'borough_trends.csv', subfolder='processed', index=False)
print("✅ Saved: borough_trends.csv")

print("\n✅ Time series analysis complete!")

## 9. Key Insights & Next Steps

### 📊 Key Findings:
- Overall trend in NYC SAT performance
- Borough-level performance changes
- Individual schools showing improvement or decline
- Performance volatility patterns

### 🔮 Forecasting (Optional Next Step):
To predict future performance, you could:
1. Use statistical methods (ARIMA, Prophet)
2. Apply machine learning regression
3. Build ensemble models

### 📈 Further Analysis:
1. **Seasonal patterns**: If you have within-year data
2. **External factors**: Correlate with policy changes, funding, etc.
3. **Intervention analysis**: Measure impact of specific programs
4. **Cohort analysis**: Track specific student groups over time